In [ ]:
TOKEN="token from quantum.ibm.com"

In [82]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
# Save an IBM Quantum account and set it as your default account.
QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token=TOKEN,
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)
 
# Load saved credentials
service = QiskitRuntimeService()

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit import transpile
from qiskit_ibm_runtime import SamplerV2 as Sampler
import math
import numpy as np

# isSimulator False run on quantum computer
# isSimulator True run on simulator
isSimulator = False

In [119]:

def generate_random_number(max_value=100, shots=1):
    """Generate a single random number between 0 and max_value."""
    # Calculate number of bits needed to represent max_value
    n_bits = math.ceil(math.log2(max_value + 1))
    
    # Create circuit
    qc = QuantumCircuit(n_bits, n_bits)
    
    # Apply Hadamard gates to create superposition on all qubits
    for i in range(n_bits):
        qc.h(i)
    
    # Measure all qubits
    qc.measure_all()

    if isSimulator:
        # Execute on the simulator
        backend = Aer.get_backend('qasm_simulator')
        job = backend.run(transpile(qc, backend), shots=shots)
        counts = job.result().get_counts(qc)
    else:
        # Initialize the service and get the backend
        service = QiskitRuntimeService()
        backend = service.least_busy(operational=True, simulator=False)

        # Create sampler and run the transpiled circuit
        sampler = Sampler(backend)
        job = sampler.run([transpile(qc, backend)], shots=shots)
        result = job.result()

        # Extracting measurement results
        counts = result[0].data.meas.get_counts()

    # Convert binary measurement results to decimal numbers
    valid_numbers = []
    for bitstring, count in counts.items():
        # Remove any spaces and reverse the bitstring (Qiskit returns bits in reverse order)
        cleaned_bitstring = bitstring.replace(" ", "")[::-1]
        number = int(cleaned_bitstring, 2)
        if number <= max_value:
            valid_numbers.extend([number] * count)
    
    return valid_numbers[0] if valid_numbers else generate_random_number(max_value, shots)

In [122]:
def generate_multiple_random_numbers(count=5, max_value=100):
    """Generate multiple random numbers in one batch"""
    # Calculate number of bits needed
    n_bits = math.ceil(math.log2(max_value + 1))
    
    # Create circuit
    qc = QuantumCircuit(n_bits, n_bits)
    
    # Apply Hadamard gates
    for i in range(n_bits):
        qc.h(i)
    
    # Measure
    qc.measure_all()

    if isSimulator:
       # Execute on the simulator
        backend = Aer.get_backend('qasm_simulator')
        job = backend.run(transpile(qc, backend), shots=count)
        counts = job.result().get_counts(qc)
    else:
        # Initialize the service and get the backend
        service = QiskitRuntimeService()
        backend = service.least_busy(operational=True, simulator=False)

        # Create sampler and run the transpiled circuit
        sampler = Sampler(backend)
        job = sampler.run([transpile(qc, backend)], shots=count)
        result = job.result()

        # Extracting measurement results
        counts = result[0].data.meas.get_counts()

    # Process results
    valid_numbers = []
    for bitstring, count in counts.items():
        # Remove any spaces and reverse the bitstring (Qiskit returns bits in reverse order)
        cleaned_bitstring = bitstring.replace(" ", "")[::-1]
        number = int(cleaned_bitstring, 2)
        if number <= max_value:
            valid_numbers.extend([number] * count)
    
    # If we don't have enough numbers, generate more
    while len(valid_numbers) < count:
        extra = generate_random_number(max_value)
        valid_numbers.append(extra)
    
    return valid_numbers[:count]


In [ ]:
def main():
    try:
        # Test single number generation
        print("\nGenerating a single random number:")
        single_num = generate_random_number()
        print(f"Random number between 0 and 100: {single_num}")

        return single_num

        # Test multiple number generation
        print("\nGenerating multiple random numbers:")
        multiple_nums = generate_multiple_random_numbers(count=5)
        print(f"Five random numbers between 0 and 100: {multiple_nums}")
        
        # Test with different range
        print("\nGenerating numbers with different range:")
        small_range_nums = generate_multiple_random_numbers(count=3, max_value=50)
        print(f"Three random numbers between 0 and 50: {small_range_nums}")
        
        # Show the quantum circuit
        print("\nQuantum Circuit used:")
        n_bits = math.ceil(math.log2(100 + 1))
        qc = QuantumCircuit(n_bits, n_bits)
        for i in range(n_bits):
            qc.h(i)
        qc.measure_all()
        print(qc)
        
        # Generate larger sample to show distribution
        print("\nGenerating 1000 numbers to show distribution...")
        large_sample = generate_multiple_random_numbers(count=1000)
        print(f"Distribution summary:")
        print(f"Min: {min(large_sample)}")
        print(f"Max: {max(large_sample)}")
        print(f"Mean: {np.mean(large_sample):.2f}")
        print(f"Standard deviation: {np.std(large_sample):.2f}")
        
        # Show frequency of first 10 numbers
        print("\nFrequency of first 10 numbers in large sample:")
        for i in range(10):
            count = large_sample.count(i)
            print(f"Number {i}: {count} times")
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage and testing
if __name__ == "__main__":
    result = main()


Generating a single random number:


KeyboardInterrupt: 


Generating a single random number:
Result(backend_name='qasm_simulator', backend_version='0.15.1', qobj_id='', job_id='d4a5ee5e-9371-45b9-9887-3b0c33196209', success=True, results=[ExperimentResult(shots=1, success=True, meas_level=2, data=ExperimentResultData(counts={'0x3c00': 1}), header=QobjExperimentHeader(creg_sizes=[['c', 7], ['meas', 7]], global_phase=0.0, memory_slots=14, n_qubits=7, name='circuit-437', qreg_sizes=[['q', 7]], metadata={}), status=DONE, seed_simulator=569318329, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 16, 'parallel_shots': 1, 'sample_measure_time': 9.84e-06, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [0, 1, 2, 3, 4, 5, 6], 'device': 'CPU', 'time_taken': 0.000123385, 'measure_sampling': True, 'num_clbits': 14, 'max_memory_mb': 32768, 'input_qubit_map': [[6, 6], [5, 5], [4, 4], [3, 3], [2, 2], [1, 1], [0, 0]], 'num_qubits': 7, 'method': 'stabilizer', 'required_memory_mb': 0, 'fusion': {'enabled': False}}, time_taken=0.000123385)], date=2024-11-22T21:54:11.582539, status=COMPLETED, header=None, metadata={'time_taken_parameter_binding': 1.7346e-05, 'max_memory_mb': 32768, 'time_taken_execute': 0.00016226, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'parallel_experiments': 1}, time_taken=0.0006778240203857422)
Random number between 0 and 100: 15


PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1, num_bits=7>), meas=BitArray(<shape=(), num_shots=1, num_bits=7>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([SliceSpan(<start='2024-11-22 17:15:21', stop='2024-11-22 17:15:39', size=1>)])}, 'version': 2})